In [1]:
import re

import json
from typing import List, Dict, Tuple, Set, Any

import numpy as np
import numpy.typing as npt


# Define Constants


In [2]:
######################################################
### Constants                                      ###
######################################################
# Base Paths
INPUT_PATH = "./data"
MODEL_PATH = "./model"

# Model File names
VANILLA_MODEL_FILENAME = "vanillamodel.txt"
AVERAGED_MODEL_FILENAME = "averagedmodel.txt"

# Class Identifiers
TRUTHFUL = "True"
DECEPTIVE = "Fake"
POSITIVE = "Pos"
NEGATIVE = "Neg"

SENTIMENT_CLASS_DICT = {1: "Pos", -1: "Neg"}

TRUTHFULNESS_CLASS_DICT = {1: "True", -1: "Fake"}

# File paths
TRAIN_FILE_PATH = f"{INPUT_PATH}/train-labeled.txt"
CLEANED_DATA_FILE_PATH = f"{INPUT_PATH}/cleaned-data.txt"
PREPROCESSED_DATA_FILE_PATH = f"{INPUT_PATH}/preprocessed-data.txt"

VANILLA_MODEL_FILE_PATH = f"{MODEL_PATH}/{VANILLA_MODEL_FILENAME}"
AVERAGED_MODEL_FILE_PATH = f"{MODEL_PATH}/{AVERAGED_MODEL_FILENAME}"

DEV_FILE_PATH = f"{INPUT_PATH}/dev-text.txt"
DEV_TAGGED_FILE_PATH = f"{INPUT_PATH}/dev-key.txt"

RANDOM_SEED = 42

DATA_COL = 3
SENTIMENT_TARGET_COL = 2
TRUTHFULNESS_TARGET_COL = 1
TEST_SIZE = 0.2


In [3]:
rng = np.random.default_rng(seed=RANDOM_SEED)
np.random.seed(RANDOM_SEED)


# Helper Functions


In [4]:
line = "07Zfn0z Fake Pos If you're looking for an elegant hotel in downtown Chicago, you have to stay here. The Ambassador East Hotel has very comfortable and beautiful large rooms and is like a home away from home. The perfect place for a business person, and if you have a small pet you can bring them too! I would give this place four stars and would definitely stay here again.\n"


In [5]:
input_regex = re.compile("(\w*) (\w*) (\w*) (.*)\n?")


In [6]:
re.match(input_regex, line).groups()


('07Zfn0z',
 'Fake',
 'Pos',
 "If you're looking for an elegant hotel in downtown Chicago, you have to stay here. The Ambassador East Hotel has very comfortable and beautiful large rooms and is like a home away from home. The perfect place for a business person, and if you have a small pet you can bring them too! I would give this place four stars and would definitely stay here again.")

In [7]:
data = []
data.append(re.match(input_regex, line).groups())
data


[('07Zfn0z',
  'Fake',
  'Pos',
  "If you're looking for an elegant hotel in downtown Chicago, you have to stay here. The Ambassador East Hotel has very comfortable and beautiful large rooms and is like a home away from home. The perfect place for a business person, and if you have a small pet you can bring them too! I would give this place four stars and would definitely stay here again.")]

In [8]:
np.array(data)[0, 1]


'Fake'

In [9]:
# Load Data
def load_data(input_file_path: str, type: str = "TRAIN") -> npt.NDArray:
    input_data = list()
    regex = "(\w*) (\w*) (\w*) (.*)\n?"
    if type == "TEST":
        regex = "(\w*) (.*)\n?"
    elif type == "LABELS":
        regex = "(\w*) (\w*) (\w*)\n?"
    input_regex = re.compile(regex)
    with open(input_file_path, mode="r") as input_file:
        for line in input_file:
            input_data.append(re.match(input_regex, line).groups())
    return np.array(input_data)


In [10]:
# Store Data
def store_data(date_file_path: str, data: npt.NDArray) -> None:
    with open(date_file_path, mode="w") as data_file:
        for row in data:
            data_file.write(f"{row[0]} {row[1]} {row[2]} {row[3]}\n")


In [11]:
# Store Model
def store_model(model_file_path: str, model_data: Any) -> None:
    with open(model_file_path, mode="w") as model_file:
        json.dump(model_data, model_file, ensure_ascii=False)


In [12]:
# Load Model
def load_model(model_file_path: str) -> npt.NDArray:
    with open(model_file_path, mode="r") as model_file:
        model_data = json.load(model_file)

    return (model_data["tf_idf_model"], model_data["sentiment_classifier"], model_data["truthfulness_classifier"])


In [13]:
# Store Predictions
def store_predictions(output_file_path: str, predictions: List[Tuple[str, str, str]]) -> None:
    with open(output_file_path, mode="w") as output_file:
        for prediction in predictions:
            output_file.write(f"{prediction[0]} {prediction[1]} {prediction[2]}\n")


In [14]:
# Calculate Scores
def calculate_scores(y_true, y_pred, title: str):
    from sklearn.metrics import classification_report

    print(f"------------------------ {title} ------------------------")
    print(classification_report(y_true, y_pred))
    print("---------------------------------------------------------")


# Load Data


In [15]:
data = load_data(TRAIN_FILE_PATH, type="TRAIN")


# Data Cleaning


In [16]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: npt.NDArray):
    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = result[i].lower()
    return result


In [17]:
def remove_html_encodings(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"&#\d+;", " ", result[i])
    return result


In [18]:
def remove_html_tags(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"<[a-zA-Z]+\s?/?>", "", result[i])
    return result


In [19]:
def remove_url(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", "", result[i])
    return result


In [20]:
def remove_html_and_url(data):
    """Function to remove
             1. HTML encodings
             2. HTML tags (both closed and open)
             3. URLs

    Args:
        data (npt.NDArray): A Numpy Array of type string

    Returns:
        _type_: npt.NDArray
    """
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        # Remove HTML encodings
        result[i] = re.sub(r"&#\d+;", "", result[i])

        # Remove HTML tags (both open and closed)
        result[i] = re.sub(r"<[a-zA-Z]+\s?/?>", "", result[i])

        # Remove URLs
        result[i] = re.sub(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", "", result[i])

    return result


In [21]:
def replace_digits_with_tag(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"\d+", " NUM ", result[i])
    return result


In [22]:
# Remove non-alphabetical characters
def remove_non_alpha_characters(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"_+|\\|[^a-zA-Z0-9\s]", " ", result[i])
    return result


In [23]:
# Remove extra spaces
def remove_extra_spaces(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"^\s*|\s\s*", " ", result[i])
    return result


In [24]:
# Expanding contractions
def fix_contractions(data: npt.NDArray):
    # TODO: Replace with custom implementation
    import contractions

    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = contraction_fixer(result[i])
    return result


In [25]:
# A dictionary containing the columns and a list of functions to perform on it in order
data_cleaning_pipeline = {
    DATA_COL: [
        to_lower,
        remove_html_encodings,
        remove_html_tags,
        remove_url,
        fix_contractions,
        remove_non_alpha_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = data.copy()

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = cleaned_data[:, col].copy()

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    cleaned_data[:, col] = temp_data.copy()


Starting: to_lower
Ended: to_lower
Starting: remove_html_encodings
Ended: remove_html_encodings
Starting: remove_html_tags
Ended: remove_html_tags
Starting: remove_url
Ended: remove_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpha_characters
Ended: remove_non_alpha_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [26]:
store_data(CLEANED_DATA_FILE_PATH, cleaned_data)


# Data Preprocessing


In [27]:
# def tokenize(data: pd.Series):
#     from nltk.tokenize import word_tokenize

#     nltk.download("punkt")

#     return data.apply(word_tokenize)


In [28]:
# def remove_stopwords(data: pd.Series):
#     """Remove stop words using the NLTK stopwords dictionary

#     Args:
#         string (str): a document

#     Returns:
#         str: a document with stopwords removed
#     """
#     from nltk.corpus import stopwords

#     nltk.download("stopwords")

#     stopwords = set(stopwords.words())

#     def remover(word_list: List[str], stopwords: Set[str]):
#         return [word for word in word_list if not word in stopwords]

#     return data.apply(lambda word_list: remover(word_list, stopwords))


In [29]:
# def lemmatize(data: pd.Series, consider_pos_tag: bool = True):
#     from nltk.corpus import wordnet
#     from nltk.stem import WordNetLemmatizer

#     nltk.download("omw-1.4")

#     # POS tagging
#     def perform_nltk_pos_tag(data: pd.Series):
#         from nltk import pos_tag

#         nltk.download("averaged_perceptron_tagger")

#         return data.apply(pos_tag)

#     # Convert POS tag to wordnet pos tags
#     def wordnet_pos_tagger(tag: str):
#         if tag.startswith("J"):
#             return wordnet.ADJ
#         elif tag.startswith("V"):
#             return wordnet.VERB
#         elif tag.startswith("N"):
#             return wordnet.NOUN
#         elif tag.startswith("R"):
#             return wordnet.ADV
#         else:
#             return None

#     lemmatizer = WordNetLemmatizer()
#     lemmatized = list()

#     if consider_pos_tag:
#         pos_tagged_data = data.copy()
#         pos_tagged_data = perform_nltk_pos_tag(data)

#         for row in pos_tagged_data:

#             lemmatized_row = list()

#             if consider_pos_tag:
#                 for word, tag in row:
#                     wordnet_pos_tag = wordnet_pos_tagger(tag)

#                     if wordnet_pos_tag is None:
#                         lemmatized_row.append(word)
#                     else:
#                         result = lemmatizer.lemmatize(word, wordnet_pos_tag)
#                         lemmatized_row.append(lemmatizer.lemmatize(word, wordnet_pos_tag))

#             lemmatized.append(lemmatized_row)
#     else:
#         for row in data:
#             lemmatized_row = list()

#             for word in row:
#                 lemmatized_row.append(lemmatizer.lemmatize(word))

#             lemmatized.append(lemmatized_row)

#     return pd.Series(lemmatized)


In [30]:
# Concatenate lemmatized sentences back into one sentence
# def concatenate(data: pd.Series):
#     return data.apply(lambda words: " ".join(words))


In [31]:
# preprocessing_pipeline = {DATA_COL: [tokenize, lemmatize, concatenate]}

# # Run the pipeline
# preprocessed_data = cleaned_data.copy()

# # Process all the cleaning instructions
# for col, pipeline in preprocessing_pipeline.items():
#     # Get the column to perform cleaning on
#     temp_data = preprocessed_data[col].copy()

#     # Perform all the cleaning functions sequencially
#     for func in pipeline:
#         print(f"Starting: {func.__name__}")

#         if func.__name__ == "lemmatize":
#             temp_data = func(temp_data, consider_pos_tag=True)
#         else:
#             temp_data = func(temp_data)

#         print(f"Ended: {func.__name__}")

#     # Replace the old column with cleaned one.
#     preprocessed_data[col] = temp_data.copy()


# Feature Extraction


In [32]:
class TfIdf:
    def __init__(self) -> None:
        self.n_docs: int = None
        self.vocab: List = list()
        self.vocab_size: int = None
        self.vocab_index: Dict[str, int] = dict()
        self.word_document_count: Dict[str, int] = dict()

    def __create_vocab__(self, documents: npt.NDArray) -> Set:
        vocab = set()

        for document in documents:
            for word in document:
                vocab.add(word)

        return list(vocab)

    def __get_word_document_count__(self, documents: npt.NDArray):
        word_document_count = dict()

        for word in self.vocab:
            word_document_count[word] = 0
            for document in documents:
                if word in document:
                    word_document_count[word] += 1

        return word_document_count

    def __term_frequency__(self, word: str, document: npt.NDArray):
        word_occurences = (document == word).sum()
        return word_occurences / self.n_docs

    def __inverse_document_frequency__(self, word: str):
        word_occurrences = 1

        if word in self.word_document_count:
            word_occurrences += self.word_document_count[word]

        return np.log(self.n_docs / word_occurrences)

    def __tf_idf__(self, document: npt.NDArray):
        tf_idf_vector = np.zeros(shape=(self.vocab_size,))
        for word in document:
            tf = self.__term_frequency__(word, document)
            idf = self.__inverse_document_frequency__(word)

            tf_idf_vector[self.vocab_index[word]] = tf * idf
        return tf_idf_vector

    def fit(self, documents: npt.NDArray):
        self.n_docs = documents.shape[0]
        self.vocab = self.__create_vocab__(documents)
        self.vocab_size = len(self.vocab)
        self.vocab_index = {word: idx for idx, word in enumerate(self.vocab)}
        self.word_document_count = self.__get_word_document_count__(documents)

    def transform(self, documents: npt.NDArray):
        tf_idf_vectors = list()
        for document in documents:
            tf_idf_vectors.append(self.__tf_idf__(document))
        return np.array(tf_idf_vectors)

    def export(self):
        return {
            "n_docs": self.n_docs,
            "vocab_size": self.vocab_size,
            "vocab": self.vocab,
            "vocab_size": self.vocab_size,
            "vocab_index": self.vocab_index,
            "word_document_count": self.word_document_count,
        }

    def load(self, tf_idf_model_data):
        self.n_docs = tf_idf_model_data["n_docs"]
        self.vocab_size = tf_idf_model_data["vocab_size"]
        self.vocab = tf_idf_model_data["vocab"]
        self.vocab_size = tf_idf_model_data["vocab_size"]
        self.vocab_index = tf_idf_model_data["vocab_index"]
        self.word_document_count = tf_idf_model_data["word_document_count"]


In [33]:
final_data = load_data(CLEANED_DATA_FILE_PATH)


In [34]:
def tokenize(data: npt.NDArray):
    tokenized_documents = list()
    for document in data:
        tokenized_documents.append(np.array(document.split()))
    return np.array(tokenized_documents, dtype=object)


In [35]:
tokenized_documents = tokenize(final_data[:, DATA_COL])
tokenized_documents[0]


array(['if', 'you', 'are', 'looking', 'for', 'an', 'elegant', 'hotel',
       'in', 'downtown', 'chicago', 'you', 'have', 'to', 'stay', 'here',
       'the', 'ambassador', 'east', 'hotel', 'has', 'very', 'comfortable',
       'and', 'beautiful', 'large', 'rooms', 'and', 'is', 'like', 'a',
       'home', 'away', 'from', 'home', 'the', 'perfect', 'place', 'for',
       'a', 'business', 'person', 'and', 'if', 'you', 'have', 'a',
       'small', 'pet', 'you', 'can', 'bring', 'them', 'too', 'i', 'would',
       'give', 'this', 'place', 'four', 'stars', 'and', 'would',
       'definitely', 'stay', 'here', 'again'], dtype='<U11')

In [36]:
# TODO: Invoke TF-IDF
tf_idf_model = TfIdf()
tf_idf_model.fit(tokenized_documents)


In [37]:
tf_idf_vectors = tf_idf_model.transform(tokenized_documents)
tf_idf_vectors.shape


(960, 7655)

In [38]:
tf_idf_model_data = tf_idf_model.export()


In [39]:
sentiment_labels = np.where(final_data[:, SENTIMENT_TARGET_COL] == POSITIVE, 1, -1)
truthfulness_labels = np.where(final_data[:, TRUTHFULNESS_TARGET_COL] == TRUTHFUL, 1, -1)


# Split Data


In [40]:
def train_test_split(data: npt.NDArray, labels: npt.NDArray, test_size: float = 0.2):
    n_max = data.shape[0]
    sample = int((1 - test_size) * n_max)

    all_idx = np.random.permutation(n_max)
    train_idx, test_idx = all_idx[:sample], all_idx[sample:]

    X_train, X_test, y_train, y_test = data[train_idx], data[test_idx], labels[train_idx], labels[test_idx]

    return X_train, X_test, y_train, y_test


In [41]:
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(
    tf_idf_vectors, sentiment_labels, TEST_SIZE
)
X_train_truthfulness, X_test_truthfulness, y_train_truthfulness, y_test_truthfulness = train_test_split(
    tf_idf_vectors, truthfulness_labels, TEST_SIZE
)


# Perceptron Models


## Vanilla Perceptron


In [42]:
class VanillaPerceptron:
    def __init__(self, n_features: int) -> None:
        self.n_features: int = n_features
        self.weights: npt.NDArray = np.zeros(shape=(self.n_features,))
        self.bias: float = 0.0

    # TODO: Can implement this ... tolerance: float, early_stopping: bool = True
    def fit(self, X: npt.NDArray, y: npt.NDArray, max_iterations: int, shuffle: bool = True):
        misclassified = 0
        n_epoch = 0

        for epoch in range(max_iterations):
            n_epoch = epoch
            print(f"Epoch #: {n_epoch + 1}")

            if shuffle:
                idxs = np.random.permutation(X.shape[0])
                X = X[idxs]
                y = y[idxs]

            misclassified = 0
            for idx, x in enumerate(X):
                target = y[idx]

                a = np.dot(x, self.weights) + self.bias

                if target * a <= 0:
                    self.weights += target * x
                    self.bias += target
                    misclassified += 1

            if misclassified == 0:
                break

    def predict(self, X: npt.NDArray):
        predictions = list()
        for x in X:
            a = np.sign(np.dot(x, self.weights) + self.bias)
            predictions.append(a)
        return np.array(predictions)

    def export(
        self,
    ):
        return {"n_features": self.n_features, "weights": self.weights.tolist(), "bias": self.bias}

    def load(self, perceptron_data: Dict[str, Any]):
        self.n_features = perceptron_data["n_features"]
        self.weights = np.array(perceptron_data["weights"])
        self.bias = perceptron_data["bias"]


## Sentiment Classification


#### Learn


In [43]:
n_features = X_train_sentiment.shape[-1]


In [44]:
vanilla_perceptron_sentiment = VanillaPerceptron(n_features)
vanilla_perceptron_sentiment.fit(X_train_sentiment, y_train_sentiment, max_iterations=100)

vanilla_perceptron_sentiment_data = vanilla_perceptron_sentiment.export()


Epoch #: 1
Epoch #: 2
Epoch #: 3
Epoch #: 4
Epoch #: 5
Epoch #: 6
Epoch #: 7
Epoch #: 8
Epoch #: 9
Epoch #: 10
Epoch #: 11
Epoch #: 12
Epoch #: 13
Epoch #: 14
Epoch #: 15
Epoch #: 16
Epoch #: 17
Epoch #: 18
Epoch #: 19
Epoch #: 20
Epoch #: 21
Epoch #: 22
Epoch #: 23
Epoch #: 24
Epoch #: 25
Epoch #: 26
Epoch #: 27
Epoch #: 28
Epoch #: 29
Epoch #: 30
Epoch #: 31
Epoch #: 32
Epoch #: 33
Epoch #: 34
Epoch #: 35
Epoch #: 36
Epoch #: 37
Epoch #: 38
Epoch #: 39
Epoch #: 40
Epoch #: 41
Epoch #: 42
Epoch #: 43
Epoch #: 44
Epoch #: 45
Epoch #: 46
Epoch #: 47
Epoch #: 48
Epoch #: 49
Epoch #: 50
Epoch #: 51
Epoch #: 52
Epoch #: 53
Epoch #: 54
Epoch #: 55
Epoch #: 56
Epoch #: 57
Epoch #: 58
Epoch #: 59
Epoch #: 60
Epoch #: 61
Epoch #: 62
Epoch #: 63
Epoch #: 64
Epoch #: 65
Epoch #: 66
Epoch #: 67
Epoch #: 68
Epoch #: 69
Epoch #: 70
Epoch #: 71
Epoch #: 72
Epoch #: 73
Epoch #: 74
Epoch #: 75
Epoch #: 76
Epoch #: 77
Epoch #: 78
Epoch #: 79
Epoch #: 80
Epoch #: 81
Epoch #: 82
Epoch #: 83
Epoch #: 84
E

#### Classify


In [45]:
y_pred_sentiment = vanilla_perceptron_sentiment.predict(X_test_sentiment)


In [46]:
calculate_scores(
    np.where(y_train_sentiment == -1, "Neg", "Pos"),
    np.where(vanilla_perceptron_sentiment.predict(X_train_sentiment) == -1, "Neg", "Pos"),
    title="Train",
)
calculate_scores(
    np.where(y_test_sentiment == -1, "Neg", "Pos"), np.where(y_pred_sentiment == -1, "Neg", "Pos"), title="Test"
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

         Neg       0.77      1.00      0.87       387
         Pos       1.00      0.70      0.82       381

    accuracy                           0.85       768
   macro avg       0.88      0.85      0.85       768
weighted avg       0.88      0.85      0.85       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

         Neg       0.57      0.99      0.73        93
         Pos       0.97      0.31      0.47        99

    accuracy                           0.64       192
   macro avg       0.77      0.65      0.60       192
weighted avg       0.78      0.64      0.60       192

---------------------------------------------------------


## Truthfulness Classification


#### Learn


In [47]:
n_features = X_train_truthfulness.shape[-1]


In [48]:
vanilla_perceptron_truthfulness = VanillaPerceptron(n_features)
vanilla_perceptron_truthfulness.fit(X_train_truthfulness, y_train_truthfulness, max_iterations=100)

vanilla_perceptron_truthfulness_data = vanilla_perceptron_truthfulness.export()


Epoch #: 1
Epoch #: 2
Epoch #: 3
Epoch #: 4
Epoch #: 5
Epoch #: 6
Epoch #: 7
Epoch #: 8
Epoch #: 9
Epoch #: 10


#### Classify


In [49]:
y_pred_truthfulness = vanilla_perceptron_truthfulness.predict(X_test_truthfulness)


In [50]:
calculate_scores(
    np.where(y_train_truthfulness == -1, "Fake", "True"),
    np.where(vanilla_perceptron_truthfulness.predict(X_train_truthfulness) == -1, "Fake", "True"),
    title="Train",
)
calculate_scores(
    np.where(y_test_truthfulness == -1, "Fake", "True"),
    np.where(y_pred_truthfulness == -1, "Fake", "True"),
    title="Test",
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       383
        True       1.00      1.00      1.00       385

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

        Fake       0.92      0.68      0.78        97
        True       0.74      0.94      0.83        95

    accuracy                           0.81       192
   macro avg       0.83      0.81      0.80       192
weighted avg       0.83      0.81      0.80       192

---------------------------------------------------------


#### Write Vanilla Models


In [51]:
vanilla_model_file_data = {
    "tf_idf_model": tf_idf_model_data,
    "sentiment_classifier": vanilla_perceptron_sentiment_data,
    "truthfulness_classifier": vanilla_perceptron_truthfulness_data,
}

store_model(VANILLA_MODEL_FILE_PATH, vanilla_model_file_data)


#### Load Vanilla Models


In [65]:
tf_idf_model_data, vanilla_perceptron_sentiment_data, vanilla_perceptron_truthfulness_data = load_model(
    VANILLA_MODEL_FILE_PATH
)


In [66]:
tf_idf_model = TfIdf()
tf_idf_model.load(tf_idf_model_data)

n_features = vanilla_perceptron_sentiment_data["n_features"]
vanilla_perceptron_sentiment = VanillaPerceptron(n_features)
vanilla_perceptron_sentiment.load(vanilla_perceptron_sentiment_data)

n_features = vanilla_perceptron_truthfulness_data["n_features"]
vanilla_perceptron_truthfulness = VanillaPerceptron(n_features)
vanilla_perceptron_truthfulness.load(vanilla_perceptron_truthfulness_data)


##### Test Loaded Model


In [67]:
tf_idf_vectors = tf_idf_model.transform(tokenized_documents)
tf_idf_vectors.shape


(960, 7655)

In [68]:
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(
    tf_idf_vectors, sentiment_labels, TEST_SIZE
)
X_train_truthfulness, X_test_truthfulness, y_train_truthfulness, y_test_truthfulness = train_test_split(
    tf_idf_vectors, truthfulness_labels, TEST_SIZE
)


In [69]:
y_pred_sentiment = vanilla_perceptron_sentiment.predict(X_test_sentiment)


In [70]:
calculate_scores(
    np.where(y_train_sentiment == -1, "Neg", "Pos"),
    np.where(vanilla_perceptron_sentiment.predict(X_train_sentiment) == -1, "Neg", "Pos"),
    title="Train",
)
calculate_scores(
    np.where(y_test_sentiment == -1, "Neg", "Pos"), np.where(y_pred_sentiment == -1, "Neg", "Pos"), title="Test"
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

         Neg       0.72      1.00      0.84       381
         Pos       1.00      0.62      0.76       387

    accuracy                           0.81       768
   macro avg       0.86      0.81      0.80       768
weighted avg       0.86      0.81      0.80       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

         Neg       0.74      0.99      0.85        99
         Pos       0.98      0.63      0.77        93

    accuracy                           0.82       192
   macro avg       0.86      0.81      0.81       192
weighted avg       0.86      0.82      0.81       192

---------------------------------------------------------


In [73]:
y_pred_truthfulness = vanilla_perceptron_truthfulness.predict(X_test_truthfulness)


In [74]:
calculate_scores(
    np.where(y_train_truthfulness == -1, "Fake", "True"),
    np.where(vanilla_perceptron_truthfulness.predict(X_train_truthfulness) == -1, "Fake", "True"),
    title="Train",
)
calculate_scores(
    np.where(y_test_truthfulness == -1, "Fake", "True"),
    np.where(y_pred_truthfulness == -1, "Fake", "True"),
    title="Test",
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

        Fake       0.99      0.95      0.97       385
        True       0.95      0.99      0.97       383

    accuracy                           0.97       768
   macro avg       0.97      0.97      0.97       768
weighted avg       0.97      0.97      0.97       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

        Fake       0.99      0.89      0.94        95
        True       0.91      0.99      0.95        97

    accuracy                           0.94       192
   macro avg       0.95      0.94      0.94       192
weighted avg       0.95      0.94      0.94       192

---------------------------------------------------------


--------------------------------------------------------- ******************\*\*\******************* -------------------------------------------------


## Averaged Perceptron


In [92]:
class AveragedPerceptron:
    def __init__(self, n_features: int) -> None:
        self.n_features = n_features
        self.weights = np.zeros(shape=(self.n_features,))
        self.bias = 0.0
        self.cache = {"weights": np.zeros(shape=(self.n_features,)), "bias": 0.0}

    # TODO: Can implement this ... tolerance: float, early_stopping: bool = True
    def fit(self, X: npt.NDArray, y: npt.NDArray, max_iterations: int, shuffle: bool = True):
        misclassified = 0
        n_epoch = 0

        c = 1
        for epoch in range(max_iterations):
            n_epoch = epoch
            print(f"Epoch #: {n_epoch + 1}")

            if shuffle:
                idxs = np.random.permutation(X.shape[0])
                X = X[idxs]
                y = y[idxs]

            misclassified = 0
            for idx, x in enumerate(X):
                target = y[idx]

                a = np.dot(x, self.weights) + self.bias

                if target * a <= 0:
                    self.weights += target * x
                    self.bias += target

                    self.cache["weights"] += target * c * x
                    self.cache["bias"] += target * c

                    misclassified += 1

            c += 1

            if misclassified == 0:
                self.weights -= (1 / c) * self.cache["weights"]
                self.bias -= (1 / c) * self.cache["bias"]
                break

    def predict(self, X: npt.NDArray):
        predictions = list()
        for x in X:
            a = np.sign(np.dot(x, self.weights) + self.bias)
            predictions.append(a)
        return np.array(predictions)

    def export(
        self,
    ):
        return {"n_features": self.n_features, "weights": self.weights.tolist(), "biases": self.bias}

    def load(self, perceptron_data: Dict[str, Any]):
        self.n_features = perceptron_data["n_features"]
        self.weights = perceptron_data["weights"]
        self.bias = perceptron_data["bias"]


### Sentiment Classification


#### Learn


In [93]:
n_features = X_train_sentiment.shape[-1]


In [94]:
averaged_perceptron_sentiment = AveragedPerceptron(n_features)
averaged_perceptron_sentiment.fit(X_train_sentiment, y_train_sentiment, max_iterations=100)

averaged_perceptron_sentiment_data = averaged_perceptron_sentiment.export()


Epoch #: 1
Epoch #: 2
Epoch #: 3
Epoch #: 4
Epoch #: 5


#### Classify


In [95]:
y_pred_sentiment = averaged_perceptron_sentiment.predict(X_test_sentiment)


In [96]:
calculate_scores(
    np.where(y_train_sentiment == -1, "Neg", "Pos"),
    np.where(averaged_perceptron_sentiment.predict(X_train_sentiment) == -1, "Neg", "Pos"),
    title="Train",
)
calculate_scores(
    np.where(y_test_sentiment == -1, "Neg", "Pos"), np.where(y_pred_sentiment == -1, "Neg", "Pos"), title="Test"
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

         Neg       1.00      1.00      1.00       381
         Pos       1.00      1.00      1.00       387

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

         Neg       0.83      0.96      0.89        99
         Pos       0.95      0.78      0.86        93

    accuracy                           0.88       192
   macro avg       0.89      0.87      0.87       192
weighted avg       0.89      0.88      0.87       192

---------------------------------------------------------


### Truthfulness Classification


#### Learn


In [97]:
n_features = X_train_truthfulness.shape[-1]


In [98]:
averaged_perceptron_truthfulness = AveragedPerceptron(n_features)
averaged_perceptron_truthfulness.fit(X_train_truthfulness, y_train_truthfulness, max_iterations=100)

averaged_perceptron_truthfulness_data = averaged_perceptron_truthfulness.export()


Epoch #: 1
Epoch #: 2
Epoch #: 3
Epoch #: 4
Epoch #: 5
Epoch #: 6
Epoch #: 7
Epoch #: 8
Epoch #: 9
Epoch #: 10
Epoch #: 11
Epoch #: 12
Epoch #: 13
Epoch #: 14
Epoch #: 15
Epoch #: 16
Epoch #: 17
Epoch #: 18
Epoch #: 19


#### Classify


In [99]:
y_pred_truthfulness = averaged_perceptron_truthfulness.predict(X_test_truthfulness)


In [100]:
calculate_scores(
    np.where(y_train_truthfulness == -1, "Fake", "True"),
    np.where(averaged_perceptron_truthfulness.predict(X_train_truthfulness) == -1, "Fake", "True"),
    title="Train",
)
calculate_scores(
    np.where(y_test_truthfulness == -1, "Fake", "True"),
    np.where(y_pred_truthfulness == -1, "Fake", "True"),
    title="Test",
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

        Fake       0.50      1.00      0.67       385
        True       0.00      0.00      0.00       383

    accuracy                           0.50       768
   macro avg       0.25      0.50      0.33       768
weighted avg       0.25      0.50      0.33       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

        Fake       0.49      1.00      0.66        95
        True       0.00      0.00      0.00        97

    accuracy                           0.49       192
   macro avg       0.25      0.50      0.33       192
weighted avg       0.24      0.49      0.33       192

---------------------------------------------------------


/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

#### Write Averaged Models


In [101]:
averaged_model_file_data = {
    "tf_idf_model": tf_idf_model_data,
    "sentiment_classifier": averaged_perceptron_sentiment_data,
    "truthfulness_classifier": averaged_perceptron_truthfulness_data,
}

store_model(AVERAGED_MODEL_FILE_PATH, averaged_model_file_data)


#### Load Averaged Models


tf_idf_model, averaged_perceptron_sentiment, averaged_perceptron_truthfulness = load_model(AVERAGED_MODEL_FILE_PATH)


In [102]:
tf_idf_vectors = tf_idf_model.transform(tokenized_documents)
tf_idf_vectors.shape


(960, 7655)

##### Test Loaded Models


In [103]:
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(
    tf_idf_vectors, sentiment_labels, TEST_SIZE
)
X_train_truthfulness, X_test_truthfulness, y_train_truthfulness, y_test_truthfulness = train_test_split(
    tf_idf_vectors, truthfulness_labels, TEST_SIZE
)


In [104]:
y_pred_sentiment = averaged_perceptron_sentiment.predict(X_test_sentiment)


In [105]:
calculate_scores(
    np.where(y_train_sentiment == -1, "Neg", "Pos"),
    np.where(averaged_perceptron_sentiment.predict(X_train_sentiment) == -1, "Neg", "Pos"),
    title="Train",
)
calculate_scores(
    np.where(y_test_sentiment == -1, "Neg", "Pos"), np.where(y_pred_sentiment == -1, "Neg", "Pos"), title="Test"
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

         Neg       0.95      0.99      0.97       380
         Pos       0.99      0.95      0.97       388

    accuracy                           0.97       768
   macro avg       0.97      0.97      0.97       768
weighted avg       0.97      0.97      0.97       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

         Neg       0.98      0.99      0.99       100
         Pos       0.99      0.98      0.98        92

    accuracy                           0.98       192
   macro avg       0.98      0.98      0.98       192
weighted avg       0.98      0.98      0.98       192

---------------------------------------------------------


In [106]:
y_pred_truthfulness = averaged_perceptron_truthfulness.predict(X_test_truthfulness)


In [107]:
calculate_scores(
    np.where(y_train_truthfulness == -1, "Fake", "True"),
    np.where(averaged_perceptron_truthfulness.predict(X_train_truthfulness) == -1, "Fake", "True"),
    title="Train",
)
calculate_scores(
    np.where(y_test_truthfulness == -1, "Fake", "True"),
    np.where(y_pred_truthfulness == -1, "Fake", "True"),
    title="Test",
)


------------------------ Train ------------------------
              precision    recall  f1-score   support

        Fake       0.51      1.00      0.68       392
        True       0.00      0.00      0.00       376

    accuracy                           0.51       768
   macro avg       0.26      0.50      0.34       768
weighted avg       0.26      0.51      0.34       768

---------------------------------------------------------
------------------------ Test ------------------------
              precision    recall  f1-score   support

        Fake       0.46      1.00      0.63        88
        True       0.00      0.00      0.00       104

    accuracy                           0.46       192
   macro avg       0.23      0.50      0.31       192
weighted avg       0.21      0.46      0.29       192

---------------------------------------------------------


/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  